In [26]:
import dill
import os
path = "./Manuales_migracion/"
manuales = []
for filename in os.listdir(path):
    with open(path + filename, 'rb') as file:
        manuales.append(dill.load(file))
for m in manuales:
    if m.title == "Manual Proceso de Migracion":
        manuales.remove(m)
        manual_general = m

In [23]:
for m in manuales:
    if m.title == "Manual Garantias":
        manuales.remove(m)
        man_gar = m

In [24]:
print(man_gar.get_index())

1 Ciclo de Migraciones
	1.1 Introducción
	1.2 Migraciones Previas
		1.2.1 Migración de Saldos Iniciales
		1.2.2 Migración de Clientes
	1.3 Generación de Estructuras de Bandejas
	1.4 Control de Calidad de Datos de Bandejas
		1.4.1 Programas de Control
		1.4.2 Estados de Control
		1.4.3 Salidas de Proceso
		1.4.4 Programas de Vuelco a Bantotal
	1.5 Secuencia de Ejecución
2 Diseño de Tablas de Bandejas
	2.1 Tabla BNJ002
3 Transacciones de Migraciones
	3.1 Migración Garantías Recibidas
		3.1.1 Productos Garantías Recibidas Predefinidos
		3.1.2 Transacción Migración Garantías Recibidas (módulo 499, transacción 210)
		3.1.3 Ajuste a la Operativa del Banco


In [54]:
import re


def parse_tables(input_string):
    # Dictionary to hold the result
    result = {}

    # Regular expression to capture table blocks and names
    table_blocks = re.findall(r'(\d+\.\d+ Tabla (\w+):.*?)(?=\d+\.\d+ Tabla |\Z)', input_string, re.DOTALL)

    for block, table_name in table_blocks:
        # Extract the description using a modified regular expression
        desc_match = re.search(r'\d+\.\d+ Tabla \w+:(.*?)(?=<table>)', block, re.DOTALL)
        description = desc_match.group(1).strip() if desc_match else "No description available"

        # Extract CSV records within <table> tags
        records_match = re.search(r'<table>\s*(.*?)\s*</table>', block, re.DOTALL)
        records = records_match.group(1) if records_match else "No records available"

        # Adding table info to the result dictionary
        result[table_name] = {
            "descripcion": description,
            "registros": records
        }

    return result


In [55]:
bandejas = {}
for manual in manuales:
    bandejas[manual.title] = parse_tables(manual.get_section("2"))

In [56]:
bandejas["Manual Cuentas y Personas"]["BJD002"]

{'descripcion': 'Almacena información relativa a todas las personas Físicas.\nTabla equivalente Bantotal: FSD002 y FSE002',
 'registros': 'NOMBRE,DESCRIPCIÓN,TIPO,DETALLE\nBD002Band,Bandeja,N (4.0),\nBD002Pais,País del Documento,N (3.0),País del documento de la persona (FST013).\nBD002Tdoc,Tipo de Documento,N (4.0),Tipo de documento de la persona (FST014).\nBD002Ndoc,Documento,C (25),Número de documento de la persona; no nulo. Dato Obligatorio.\nBD002Ape1,1er. Apellido,C (30),Primer Apellido.  Dato obligatorio.\nBD002Ape2,2o. Apellido,C (30),Segundo Apellido. Dato no obligatorio.\nBD002Nom1,1er. Nombre,C (25),Primer Nombre. Dato obligatorio.\nBD002Nom2,2do. Nombre,C (25),Segundo Nombre.  Dato no obligatorio.\nBD002Ebco,Empleado del Banco,C (1),Si es empleado del Banco ‘S’ o ‘N’.\nBD002Fibc,F/Ingreso al Banco,D,En caso de ser empleado; la fecha en que ingresó al Banco la cual debe ser menor al día de migración; si no es empleado; esta fecha debe ser nula.\nBD002Cant,Sexo,C (1),‘F’ femen

In [57]:
# guardo el diccionario en un archivo dill

with open('bandejas.dill', 'wb') as file:
    dill.dump(bandejas, file)


# Y lo levanto
with open('bandejas.dill', 'rb') as file:
    bandejas = dill.load(file)